In [24]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt

In [25]:
#reading data from csv file
geodata = pd.read_csv('geo_census.csv')
geodata.head()

,d,s,p,pop,hhs,fem_hhs,long,lat,km2,area,...,kitch_open_space_per_km,pop_density,hhs_density,pop_per_hhs,T123_per_sqkm,not_T123_per_sqkm,T123_per_person,not_T123_per_person,geometry,centroid
0,WAKISO,WAKISO,KYEBANDO,61051,14763,21.3,32.513362,0.356149,10.501383,10.412916,...,490.22,5813.62,1405.81,4.14,10.85,9.75,0.19,0.17,"POLYGON((32.5252889374246 0.334884258953588, 3...",POINT(32.5133620505223 0.356148577052009)
1,MUKONO,GOMA DIVISION,NANTABULIRWA,22241,4993,24.1,32.700915,0.333435,36.709773,36.376969,...,35.28,605.86,136.01,4.45,3.44,3.01,0.57,0.50,"POLYGON((32.7267447270215 0.326316235088979, 3...",POINT(32.7009147194484 0.333434596471355)
2,KAMPALA,NAKAWA DIVISION,KIWATULE,16915,4497,32.2,32.626714,0.368906,2.907240,2.881629,...,454.04,5818.23,1546.83,3.76,13.83,11.30,0.24,0.19,"POLYGON((32.6233900600041 0.357431522624744, 3...",POINT(32.6267137789408 0.368905588996633)
3,KAMPALA,NAKAWA DIVISION,LUZIRA PRISONS,13803,1344,19.0,32.640179,0.298033,1.800534,1.784589,...,1.67,7666.06,746.45,10.27,9.14,9.14,0.12,0.12,"POLYGON((32.6390124439034 0.287753994240262, 3...",POINT(32.6401789189264 0.298033199468685)
4,KAMPALA,NAKAWA DIVISION,NTINDA,5257,1356,33.6,32.618145,0.346663,2.693763,2.670111,...,88.35,1951.55,503.39,3.88,15.48,12.29,0.79,0.63,"POLYGON((32.6076705687482 0.352370728818616, 3...",POINT(32.6181453501911 0.346662584687354)


In [26]:
#Renaming some columns for better understandability
geodata.rename(columns = {'d': 'district', 's': 'subcounty', 'p':'parish', 'pop':'population', 'hhs':'households'}, inplace = True)
geodata.head()

,district,subcounty,parish,population,households,fem_hhs,long,lat,km2,area,...,kitch_open_space_per_km,pop_density,hhs_density,pop_per_hhs,T123_per_sqkm,not_T123_per_sqkm,T123_per_person,not_T123_per_person,geometry,centroid
0,WAKISO,WAKISO,KYEBANDO,61051,14763,21.3,32.513362,0.356149,10.501383,10.412916,...,490.22,5813.62,1405.81,4.14,10.85,9.75,0.19,0.17,"POLYGON((32.5252889374246 0.334884258953588, 3...",POINT(32.5133620505223 0.356148577052009)
1,MUKONO,GOMA DIVISION,NANTABULIRWA,22241,4993,24.1,32.700915,0.333435,36.709773,36.376969,...,35.28,605.86,136.01,4.45,3.44,3.01,0.57,0.50,"POLYGON((32.7267447270215 0.326316235088979, 3...",POINT(32.7009147194484 0.333434596471355)
2,KAMPALA,NAKAWA DIVISION,KIWATULE,16915,4497,32.2,32.626714,0.368906,2.907240,2.881629,...,454.04,5818.23,1546.83,3.76,13.83,11.30,0.24,0.19,"POLYGON((32.6233900600041 0.357431522624744, 3...",POINT(32.6267137789408 0.368905588996633)
3,KAMPALA,NAKAWA DIVISION,LUZIRA PRISONS,13803,1344,19.0,32.640179,0.298033,1.800534,1.784589,...,1.67,7666.06,746.45,10.27,9.14,9.14,0.12,0.12,"POLYGON((32.6390124439034 0.287753994240262, 3...",POINT(32.6401789189264 0.298033199468685)
4,KAMPALA,NAKAWA DIVISION,NTINDA,5257,1356,33.6,32.618145,0.346663,2.693763,2.670111,...,88.35,1951.55,503.39,3.88,15.48,12.29,0.79,0.63,"POLYGON((32.6076705687482 0.352370728818616, 3...",POINT(32.6181453501911 0.346662584687354)


### Converting to GeoDataFrame then to Geojson

In [17]:
geodata.dropna(inplace=True)

In [22]:
# Converting to a Geodataframe
geometry = geodata['centroid'].map(wkt.loads)
geodata = geodata.drop('centroid', axis=1)
crs = {'init': 'epsg:4326'}
geodata_gdf = gpd.GeoDataFrame(geodata, geometry = geometry)

In [23]:
geodata_gdf.to_file(r'test.geojson', driver='GeoJSON')

### Querying geodata in Mongodb

In [1]:
#creating a polygon to be used for testing
import geopandas as gpd
from shapely.geometry import Polygon

coords = [(32.506, 0.314), (32.577, 0.389), (32.609, 0.392), (32.641, 0.362), (32.582, 0.266), (32.506, 0.314)]
myPolygon = Polygon(coords)

In [2]:
from shapely.geometry import Point

long = 32.58601658
lat = 0.31274167
myPoint = Point(long, lat)

In [3]:
print(myPolygon.contains(myPoint))

True


In [4]:
query = {'geometry':
         {'$geoWithin':
          {'$polygon': [[ 32.506, 0.314], [32.577, 0.389], [32.609, 0.392], [32.641, 0.362], [32.582, 0.266], 
                      [32.506, 0.306]]
          }
         }
        }
projection = { '_id': 0}

In [5]:
def connect_mongo(db_name):
    '''
    Connects to local MongoDB
    '''
    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017/")
    db = client[db_name]
    return db

In [6]:
db = connect_mongo('geocensus_db')

In [7]:
records = db.test.find(query, projection)

In [8]:
records_list = list(records)

In [9]:
len(records_list)

74